In [1]:
import os
import json
import time
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from tavily import TavilyClient
from langchain import hub
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel
from langchain.agents import initialize_agent, AgentType
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_community.tools.tavily_search.tool import TavilySearchResults

class Queries(BaseModel):
    queries: List[str]
# Set up the LLM and Tavily API keys
OPENAI_API_KEY =  os.environ["OPENAI_API_KEY"]
TAVILY_API_KEY = os.environ["TAVILY_API_KEY"]
os.environ["TAVILY_API_KEY"] = os.environ["TAVILY_API_KEY"]
model = ChatOpenAI(model_name="gpt-4", api_key=OPENAI_API_KEY, temperature=0.7)
MODEL_NAME = "gpt-4"
TEMPERATURE = 0.7
#initialize wrapper
search = TavilySearchAPIWrapper()

#Set up tool
tavily_tool = TavilySearchResults(api_wrapper=search)

llm = ChatOpenAI(model_name="gpt-4", api_key=OPENAI_API_KEY, temperature=0.7)
model = ChatOpenAI(model_name="gpt-4", api_key=OPENAI_API_KEY, temperature=0.7)

prompt = hub.pull("hwchase17/structured-chat-agent")

#agent_chain
agent_chain = initialize_agent(
    tools=[tavily_tool],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

C:\Users\ethan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [2]:
pip install langgraph.graph 


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langgraph.graph (from versions: none)
ERROR: No matching distribution found for langgraph.graph


In [3]:
class Queries(BaseModel):
    queries: List[str]

In [4]:
from tavily import TavilyClient

class SearchAgent:
    def __init__(self, agent_chain):
        self.agent_chain = agent_chain
#searches for viral marketing vids.
    def search_trending_videos_and_memes(self, query):
        #query agent_chain
        response = self.agent_chain.run({'input': query})

        #check if string
        if isinstance(response, str):
            response = {'output': response}

        # Ensure response has 'output' key
        if 'output' in response:
            return response['output']
        else:
            return "No relevant information found."
    #makes output understandable for next agent
    def prepare_output_for_next_agent(self, query):
        # recieve output from the search function
        search_output = self.search_trending_videos_and_memes(query)

        prompt = f"{search_output}"

        return prompt

    def run(self, input_data):
        query = input_data.get('search_prompt', '')
        return {'prompt': self.prepare_output_for_next_agent(query)}

#example usage
if __name__ == "__main__":

    tavily_client = TavilyClient(api_key=TAVILY_API_KEY)


    agent_chain = initialize_agent(
        tools=[tavily_tool],
        llm=llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
    )

    search_agent = SearchAgent(agent_chain)

    input_data = {'search_prompt': "What are trending videos and memes on TikTok for marketing bath soap? You must thoroughly describe the contents of the videos that have gone viral due to funny memes for marketing."}

    final_state = search_agent.run(input_data)

    print(final_state)



C:\Users\ethan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: Since the user is asking for current trending videos and memes on TikTok specifically for marketing bath soap, I will need to use the tavily_search_results_json tool to gather the latest information.
Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": {
    "query": "Trending TikTok videos and memes for marketing bath soap"
  }
}
```
Observation: [{'url': 'https://www.tiktok.com/tag/trendingsoap', 'content': 'trending soap | 62.1M views. Watch the latest videos about #trendingsoap on TikTok.'}, {'url': 'https://www.tiktok.com/tag/marketingmemes', 'content': 'marketing memes | 2880 posts Watch the latest videos about #marketingmemes on TikTok.'}, {'url': 'https://www.tiktok.com/tag/soapmemes', 'content': 'soap memes | 66.7K people have watched this. Watch short videos about #soapmemes on TikTok.'}, {'url': 'https://ads.tiktok.com/business/creativecenter/top-products/pc/en', 'content': 'Explore Top and viral TikTok pr

In [5]:
class AdCreatorAgent:
    def __init__(self):
        self.llm = model  # Assuming 'model' is defined somewhere
        self.WRITER_PROMPT = """
        You are an expert ad videographer tasked with creating a detailed plan for a social media video ad.

        Generate a detailed plan for a social media video ad for '{product_name}' based on the idea:
        Ad Idea: {ad_idea}

        Include:
        - Videography plan (camera angles, lighting, locations) specifically tailored for '{product_name}'
        - Actions and scenes (step-by-step description of the actions in the video) related to the ad idea
        - Any dialogues or text overlays suitable for promoting '{product_name}'
        - How to creatively relate '{product_name}' to current social media trends
        """

    def generate_ad_ideas(self, ad_ideas):
        generated_ad_ideas = []

        for ad_idea in ad_ideas:
            if isinstance(ad_idea, str):
                ad_idea = {'ad_idea': ad_idea}  #convert string to dictionary 
            product_name = ad_idea.get('product_name', 'Product')  # Default product name
            prompt = self.WRITER_PROMPT.format(product_name=product_name, ad_idea=ad_idea['ad_idea'])
            response = self.llm.invoke([SystemMessage(content=prompt)])
            generated_ad_ideas.append(response.content.strip())

        return generated_ad_ideas

    def run(self, input_data):
        ad_ideas = input_data.get('ad_ideas', [])

        generated_ad_ideas = self.generate_ad_ideas(ad_ideas)
        
        #prep output data
        output_data = {
            'ad_ideas': generated_ad_ideas
        }
        
        #add logic to handle 'approved_content'
        if input_data.get('human_approved'):
            output_data['approved_content'] = "Example approved content text"  #replace with actual content
        
        return output_data  #return the generated ad ideas and other outputs


# if __name__ == "__main__":
#     ad_creator_agent = AdCreatorAgent()

#     #example ideas
#     ad_ideas = [
#         {"ad_idea": "Introduce the soap with a catchy jingle and demonstrate its benefits in a quick, engaging manner."},
#         {"ad_idea": "Tell a short story about how the soap transformed someone's skin and life, using emotional storytelling."},
#         {"ad_idea": "Include a series of quick testimonials followed by a call to action to buy the soap."}
#     ]

#     #run the agent
#     input_data = {'ad_ideas': ad_ideas, 'human_approved': True}  # Simulating human approval
#     result = ad_creator_agent.run(input_data)

#     print("Generated Ad Ideas:")
#     for i, ad_idea_details in enumerate(result['ad_ideas'], 1):
#         print(f"Ad Idea {i}:")
#         print(ad_idea_details)

In [6]:
import os
from langchain_openai import ChatOpenAI

class ScriptAgent:
    def __init__(self, llm):
        self.llm = llm
        self.prompt_template = "You are a script assistant tasked with writing a comedy sketch. Incorporate unexpected humor elements and funny memes into the script."

    def generate_script(self):
        try:
            # Generate script using the provided llm (ChatOpenAI instance)
            response = self.llm(self.prompt_template).content.strip()
            return response

        except Exception as e:
            print(f"Error generating script: {e}")
            return ""

# # Example usage
# if __name__ == "__main__":
#     # Initialize OpenAI LLM with your API key and settings
#     OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
#     llm = ChatOpenAI(model_name="gpt-4", api_key=OPENAI_API_KEY, temperature=0.7)

#     # Initialize ScriptAgent with the llm instance
#     agent = ScriptAgent(llm)

#     # Generate the script based on the predefined prompt template
#     generated_script = agent.generate_script()

#     # Print the generated script
#     print("Generated Script:")
#     print(generated_script)


In [7]:
import os
from langgraph.graph import StateGraph
from langchain_core.messages import SystemMessage
from langchain.agents import AgentType, initialize_agent
from langchain_openai import ChatOpenAI

class SEOAgent:
    def __init__(self, api_key, model_name, temperature):
        self.llm = ChatOpenAI(model_name=model_name, api_key=api_key, temperature=temperature)

    def generate_seo_tags(self, ad_ideas):
        prompt = f"""
        You are tasked with finding the best tags for optimal visibility for the following ad ideas:
        
        {ad_ideas}
        """
        messages = [SystemMessage(content=prompt)]
        response = self.llm.invoke(messages)
        return response.content.strip()

    def run(self, initial_state):
        print("Initial State:")
        print(initial_state)

        # Generate SEO tags for the provided ad ideas
        seo_tags = self.generate_seo_tags(initial_state['ad_ideas'])

        # Update state with SEO tags
        initial_state['seo_tags'] = seo_tags
        return initial_state



# # Example usage:
# if __name__ == "__main__":
#     # Replace with your OpenAI API key and model details
#     MODEL_NAME = "gpt-4"
#     TEMPERATURE = 0.7

#     # Initialize SEOAgent
#     seo_agent = SEOAgent(api_key=OPENAI_API_KEY, model_name=MODEL_NAME, temperature=TEMPERATURE)

#     # Define initial state with ad ideas
#     initial_state = {
#         'ad_ideas': [
#             "Introduce the soap with a catchy jingle and demonstrate its benefits in a quick, engaging manner.",
#             "Tell a short story about how the soap transformed someone's skin and life, using emotional storytelling.",
#             "Include a series of quick testimonials followed by a call to action to buy the soap."
#         ]
#     }

#     # Run the SEOAgent to generate SEO tags
#     final_state = seo_agent.run(initial_state)

#     # Display final state with SEO tags
#     print("\nFinal State with SEO Tags:")
#     print(final_state)

In [8]:
REFLECTION_PROMPT = """You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""

class ScriptCritiqueAgent:
    def __init__(self):
        self.llm = model

    def critique_script(self, script):
        prompt = f"""
        You are a marketing expert. Critique the following marketing script for a bath soap commercial, focusing on appropriate behavior and entertainment value:
        {script}
        """
        messages = [SystemMessage(content=prompt)]
        response = self.llm.invoke(messages)
        return response.content.strip()

    def run(self, input_data):
        critique_results = []
        for script in input_data['ad_ideas']:
            critique_result = self.critique_script(script)
            critique_results.append({'script_critique_result': critique_result})
        return {'script_critique_results': critique_results}

# if __name__ == "__main__":
#     script_critique_agent = ScriptCritiqueAgent()
#     scripts = [
#         "Introduce the soap with a catchy jingle and demonstrate its benefits in a quick, engaging manner.",
#         "Tell a short story about how the soap transformed someone's skin and life, using emotional storytelling.",
#         "Include a series of quick testimonials followed by a call to action to buy the soap."
#     ]
    
#     critique_results = script_critique_agent.run({'ad_ideas': scripts})
#     print("Script Critique Results:")
#     for i, result in enumerate(critique_results['script_critique_results'], 1):
#         print(f"Script {i}: {result['script_critique_result']}")


In [9]:
class AdCritiqueAgent:
    def __init__(self):
        self.llm = model  

    def critique_videography(self, ad_idea):
        prompt = f"""
        You are a professional videographer. Critique the following videography plan for a social media video ad. Focus on camera angles, lighting, locations, and overall visual appeal:
        {ad_idea}

        Provide detailed feedback on:
        - Strengths of the videography plan
        - Areas for improvement
        - Suggestions for better camera angles, lighting, and locations
        """
        messages = [SystemMessage(content=prompt)]
        response = self.llm.invoke(messages)
        return response.content.strip()

    def run(self, input_data):
        critique_results = []
        if 'ad_ideas' in input_data:
            for ad_idea in input_data['ad_ideas']:
                critique_result = self.critique_videography(ad_idea)
                critique_results.append({'videography_critique_result': critique_result})
        else:
            print("'ad_ideas' key not found in input_data. Cannot run critique.")
        return {'videography_critique_results': critique_results}

# if __name__ == "__main__":

#     # Example meme ideas for the ad
#     meme_ideas = [
#         "A trending dance challenge involving the product",
#         "A humorous skit about common problems solved by the product",
#         "A remix of a popular meme format featuring the product"
#     ]

#     #run the agent
#     input_data = {'ad_ideas': meme_ideas}  # Ensure 'ad_ideas' is correctly populated
#     ad_critique_agent = AdCritiqueAgent()
#     critique_result = ad_critique_agent.run(input_data)

#     # # Print the critique results if 'ad_ideas' were found
#     # if critique_result['videography_critique_results']:
#     #     print("Videography Critique Results:")
#     #     for i, critique in enumerate(critique_result['videography_critique_results'], 1):
#     #         print(f"Critique {i}: {critique['videography_critique_result']}")


In [10]:
pip install langgraph

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install langchain-langgraph

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langchain-langgraph (from versions: none)
ERROR: No matching distribution found for langchain-langgraph


In [13]:
import os
import networkx as nx
from langchain_openai import ChatOpenAI

class Node:
    def __init__(self, func):
        self.func = func
        self.next_node = None
        self.conditional_edges = {}

    def set_next(self, node):
        self.next_node = node

    def add_conditional_edge(self, key, condition, transition):
        self.conditional_edges[key] = {'condition': condition, 'transition': transition}

    def execute(self, state):
        new_state = self.func(state)
        state.update(new_state)  # Update state with new data
        
        # Check conditional edges
        for key, edge in self.conditional_edges.items():
            if edge['condition'](state):
                if edge['transition'] == 'next':
                    return self.next_node.execute(state) if self.next_node else state
                else:
                    return edge['transition']
        
        # Default if no condition matches
        if self.next_node:
            return self.next_node.execute(state)
        return state

class HumanApprovalAgent:
    def run(self, state):
        print("Reviewing the current state for human approval:")
        print(state)

        if 'created_ads' in state:
            for ad in state['created_ads']:
                ad_idea = ad['ad_idea']
                if ad_idea in state.get('script_critique_results', []):
                    continue
                print(f"Feedback needed for: {ad_idea}")
                critique = input("Enter critique (or 'skip' to continue): ").strip()
                if critique.lower() == 'skip':
                    continue
                state.setdefault('script_critique_results', []).append({'ad_idea': ad_idea, 'critique': critique})

        # Simulate human approval (replace with actual human interaction logic)
        if 'human_approved' not in state or not state['human_approved']:
            print("Ad ideas need approval.")
            approve = input("Do you approve these ad ideas? (yes/no): ").strip().lower()
            if approve == 'yes':
                state['human_approved'] = True
            else:
                state['human_approved'] = False

        return state

class ScriptAgent:
    def __init__(self, llm):
        self.llm = llm
        self.prompt_template = "You are a script assistant tasked with writing a comedy sketch. Incorporate unexpected humor elements and funny memes into the script."

    def generate_script(self, state):
        try:
            # Generate script using the provided llm (ChatOpenAI instance)
            response = self.llm(self.prompt_template).content.strip()
            state['generated_script'] = response
            return state

        except Exception as e:
            print(f"Error generating script: {e}")
            return state

class StateGraph:
    def __init__(self, state_schema):
        self.state_schema = state_schema
        self.entry_point = None

    def set_entry_point(self, node):
        self.entry_point = node

    def run(self, initial_state):
        if not self.entry_point:
            raise ValueError("Entry point is not set.")
        return self.entry_point.execute(initial_state)

class MultiAgent:
    def __init__(self, api_key, model_name, temperature):
        # Initialize the nodes
        self.create_ad_node = Node(AdCreatorAgent().run)
        self.critique_script_node = Node(ScriptCritiqueAgent().run)
        self.critique_videography_node = Node(AdCritiqueAgent().run)
        self.seo_node = Node(SEOAgent(api_key, model_name, temperature).run)
        self.human_approval_node = Node(HumanApprovalAgent().run)

        # Initialize the ScriptAgent
        self.script_agent = ScriptAgent(ChatOpenAI(model_name=model_name, api_key=api_key, temperature=temperature))
        self.script_node = Node(self.script_agent.generate_script)

        # Create state graph
        state_schema = {}  # Define a simple state schema, modify as needed
        self.graph = StateGraph(state_schema)
        self.graph.set_entry_point(self.create_ad_node)

        # Define the workflow using networkx
        self.workflow = nx.DiGraph()

        # Add nodes for each agent
        self.workflow.add_node("create_ad", func=self.create_ad_node.execute)
        self.workflow.add_node("critique_script", func=self.critique_script_node.execute)
        self.workflow.add_node("critique_videography", func=self.critique_videography_node.execute)
        self.workflow.add_node("seo", func=self.seo_node.execute)
        self.workflow.add_node("human_approval", func=self.human_approval_node.execute)
        self.workflow.add_node("script", func=self.script_node.execute)

        # Set up edges
        self.workflow.add_edge('create_ad', 'critique_script')
        self.workflow.add_edge('critique_script', 'critique_videography')
        self.workflow.add_edge('critique_videography', 'seo')
        self.workflow.add_edge('seo', 'human_approval')
        self.workflow.add_edge('script', 'create_ad')

        # Define conditional edges
        self.conditional_edges = {
            'create_ad': ("critique_script", lambda state: state.get('ad_ideas')),
            'critique_script': ("critique_videography", lambda state: state.get('script_critique_results')),
            'critique_videography': ("seo", lambda state: state.get('videography_critique_results')),
            'seo': ("human_approval", lambda state: state.get('seo_results')),
            'human_approval': ("end", lambda state: state.get('human_approved')),
            'script': ("create_ad", lambda state: 'generated_script' in state)
        }

    def run(self, initial_state):
        state = initial_state.copy()
        current_node = 'create_ad'

        while current_node and current_node != 'end':
            func = self.workflow.nodes[current_node]['func']
            state = func(state)

            # Check for conditional edges
            if current_node in self.conditional_edges:
                next_node, condition = self.conditional_edges[current_node]
                if condition(state):
                    current_node = next_node
                else:
                    next_nodes = list(self.workflow.successors(current_node))
                    current_node = next_nodes[0] if next_nodes else None
            else:
                next_nodes = list(self.workflow.successors(current_node))
                current_node = next_nodes[0] if next_nodes else None

        return state

if __name__ == "__main__":
    initial_state = {
        'ad_ideas': [
            "There's a user with the handle @buttraders who shares content about soap making and cutting, including tips, tutorials, and trends, with a focus on how to start and grow a soap business. Their videos fall under various hashtags like #soapmaking, #soapcutting, #soapasmr, and #handmadesoap. There's also a mention of a viralsoap tag on TikTok, but the specific content isn't detailed. For the most current and specific examples, it might be helpful to look directly on the TikTok platform."
        ]
    }

    api_key = OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
    model_name = "gpt-4"
    temperature = 0.7

    multi_agent = MultiAgent(api_key, model_name, temperature)
    final_state = multi_agent.run(initial_state)
    print(final_state)


Initial State:
{'ad_ideas': ['Product: \'Buttra Soap\'\n\nVideography Plan:\n\nOur ad will mimic the aesthetic of a high-quality, ASMR-style homemade video, like those popular on TikTok, to capture the attention of our target audience. We will utilize a combination of close-up and medium shots with a soft natural light to highlight the product\'s organic nature.\n\n1. Camera Angles: The video will primarily use top-down and close-up shots to emphasize the process of soap making, the soap\'s texture, and the product details. A few medium shots will be used to showcase the environment to give it a personal touch.\n\n2. Lighting: We\'ll use soft natural lighting to make the video look as organic and authentic as possible. It will accentuate the natural colors of the product and create a warm, inviting atmosphere.\n\n3. Location: The video will be filmed in a well-lit, cozy, and rustic kitchen setting to align with the homemade and natural feel of the product.\n\nActions and Scenes:\n\n1. 

In [ ]:
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

class AdOptimizationAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0   # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        #neural net for learning Model
        model = Sequential()
        model.add(Dense(64, input_dim=self.state_size, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        state = np.reshape(state, (1, self.state_size))  # Ensure correct shape
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        # Check if enough samples
        if len(self.memory) < batch_size:
            return
        
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                next_state = np.reshape(next_state, (1, self.state_size))
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            state = np.reshape(state, (1, self.state_size))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

    def best_parameters(self, state):
        action = self.act(state)
        if action == 0:
            return {"bid": "decrease", "creative": "no change"}
        elif action == 1:
            return {"bid": "increase", "creative": "no change"}
        # Add more actions as needed
        else:
            return {"bid": "no change", "creative": "change to creative strategy"}

    def optimize_profit(self, state_data, action_data, reward_data, next_state_data, done_data):
        # Evaluate the most optimized profit based on historical data
        max_profit = float('-inf')
        best_params = None
        
        for state, action, reward, next_state, done in zip(state_data, action_data, reward_data, next_state_data, done_data):
            state = np.reshape(state, (1, self.state_size))
            action = self.act(state)
            next_state = np.reshape(next_state, (1, self.state_size))
            predicted_reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            
            if predicted_reward > max_profit:
                max_profit = predicted_reward
                best_params = {"state": state.flatten().tolist(), "action": action, "predicted_profit": predicted_reward}
        
        return best_params

#example
state_size = 4  
action_size = 2  

#create an instance
agent = AdOptimizationAgent(state_size, action_size)

#example state array
state = np.array([0.75, 0.22, 0.45, 30])  # CTR, Bid Amount, Engagement Rate, CPC

#get best params
best_params = agent.best_parameters(state)
print("Best Parameters:", best_params)

#training loop
EPISODES = 10  # Number of episodes
STEPS_PER_EPISODE = 100  # Steps per episode

# ex data
state_data = np.array([
    [0.75, 0.22, 0.45, 30],
    [0.68, 0.18, 0.42, 28],
    [0.82, 0.25, 0.50, 32],
    [0.79, 0.23, 0.48, 31]
])
action_data = np.array([1, 0, 1, 0])  # 1 for Bid Up, 0 for Bid Down
reward_data = np.array([100, -50, 80, -30])  #example rewards
next_state_data = np.array([
    [0.78, 0.25, 0.48, 35],
    [0.69, 0.20, 0.40, 25],
    [0.81, 0.28, 0.52, 34],
    [0.75, 0.21, 0.45, 29]
])
done_data = np.array([False, False, False, True])  #example done flags

for episode in range(EPISODES):
    state = state_data[0]  #start with first state
    total_reward = 0

    for step in range(STEPS_PER_EPISODE):
        action = agent.act(state)
        next_state = next_state_data[step % len(next_state_data)]  #cycling through next states
        reward = reward_data[step % len(reward_data)]  #cycling through rewards
        done = done_data[step % len(done_data)]  #cycling through done flags

        agent.remember(state, action, reward, next_state, done)
        agent.replay(batch_size=32)  #replay to train the agent
        state = next_state
        total_reward += reward

        if done:
            break

    print(f"Episode {episode+1}/{EPISODES}, Total Reward: {total_reward}")

# save final trained model
agent.save("ad_optimization_model.weights.h5")

# optimize profit
optimized_params = agent.optimize_profit(state_data, action_data, reward_data, next_state_data, done_data)
print("Optimized Parameters:", optimized_params)


Best Parameters: {'bid': 'increase', 'creative': 'no change'}
Episode 1/10, Total Reward: 100
Episode 2/10, Total Reward: 100
Episode 3/10, Total Reward: 100
Episode 4/10, Total Reward: 100
Episode 5/10, Total Reward: 100
Episode 6/10, Total Reward: 100
Episode 7/10, Total Reward: 100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello():
    return 'Hello, World!'

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1